# Census: Downloading state legislative districts
---

I used these links and copied the url from downloading the zip file. It's the same format for each state so you only need to download it once to get the url.
- <u>[Upper chamber (for single state)](https://catalog.data.gov/dataset/tiger-line-shapefile-current-state-alabama-state-legislative-district-sld-upper-chamber)</u>
- <u>[Lower chamber (for single state)](https://catalog.data.gov/dataset/tiger-line-shapefile-current-state-alabama-state-legislative-district-sld-lower-chamber)</u>


In [ ]:
import os
import io
import requests
import zipfile
import tempfile
import shutil
import geopandas as gpd
import ibis
from ibis import _
from cng.utils import *
from cng.h3 import *

duckdb_install_h3()
con = ibis.duckdb.connect(extensions = ["spatial", "h3"])
con.raw_sql("SET THREADS=100;")
set_secrets(con)

bucket = "public-census"
s3_prefix = "2024/sld"

#get fips code for each state
fips_url = 'https://www2.census.gov/geo/docs/reference/codes2020/national_state2020.txt'
fips_codes = con.read_csv(fips_url).filter(_.STATE.notin(['AS','GU','MP','PR','UM','VI'])).select("STATEFP").execute().values.flatten().tolist()


In [ ]:
def shape_to_parquet(fips_codes, s3_prefix, bucket, chamber):
    '''
    download zip/shape file -> convert to parquet -> save to minio 
    '''
    chamber_lowercase = chamber.lower()  # sldl or sldu
    for state in fips_codes:

        # 1 shape file per state, need to get each one with fips code 
        url = f"https://www2.census.gov/geo/tiger/TIGER2024/{chamber}/tl_2024_{state}_{chamber_lowercase}.zip"
        print(f"Processing state {state}: {url}")
    
        # download zip 
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to download for state {state}")
            continue
    
        zip_bytes = io.BytesIO(response.content)
    
        # extract zip in temp directory 
        with tempfile.TemporaryDirectory() as temp_dir:
            with zipfile.ZipFile(zip_bytes) as zf:
                shapefile_prefix = f"tl_2024_{state}_{chamber_lowercase}"
                shp_name = f"{shapefile_prefix}.shp"

                if shp_name not in zf.namelist():
                    print(f"Shapefile not found for {state}")
                    continue

                zf.extractall(temp_dir)

    
        shp_path = os.path.join(temp_dir, shp_name)
        gdf = con.read_geo(shp_path)
        
        # ibis isn't good at crs, so use gpd 
        # d = gpd.read_file(shp_path)
        # print(d.crs)
    
        # convert to parquet
        parquet_name = f"{shapefile_prefix}.parquet"
        chamber_folder = "lower" if chamber.upper() == "SLDL" else "upper"
        parquet_path = f"s3://{bucket}/{s3_prefix}/{chamber_folder}/{parquet_name}"
        gdf.to_parquet(parquet_path)
        return 

In [ ]:
# lower chamber 
chamber = 'SLDL'
#DC and Nebraska dont have a lower chamber 
fips_codes.remove('11')
fips_codes.remove('31') 

shape_to_parquet(fips_codes, s3_prefix, bucket, chamber)

In [ ]:
# upper chamber 
chamber = 'SLDU'
shape_to_parquet(fips_codes, s3_prefix, bucket, chamber)